In [365]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import random
import math
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression
from scipy.signal import find_peaks


In [443]:
def HurstFunction(rawBOLD, TR):
    '''This function will analyze a rs-BOLD raw signal and output 
    the appropriate Hurst exponent according to the procedure of 
    Eke et al. (Eur J Physiol (2000) 439:403?415)'''
    
    assert (len(rawBOLD) !=0), 'rawBOLD is empty'
    assert (TR > 0), 'TR is less than zero'
    
    #INITIALIZATION
    Hurst = None
    abort = False
    
    #PARAMETERS
    rawBOLD = rawBOLD[2:]
    fs = 1/TR #sampling frequency
    print('Sampling frequency =', fs)
    n = len(rawBOLD) #number of timepoints
    print('Number of timepoints =', n)

    #NORMALIZE TIME SERIES
    mean_rawBOLD = np.mean(rawBOLD)
    rawBOLD_norm = rawBOLD - mean_rawBOLD #rawBOLD normalized by subtracting mean from every data point
    print('Mean rawBOLD =', mean_rawBOLD)
    
   #PARABOLIC WINDOW
    N = len(rawBOLD_norm)
    W = np.zeros((N,1))
    for j in np.arange(0,N):
        W[j] = 1 - np.power((2*(j + 1)/(N+1)-1),2)
    #multiply each normalized value by parabolic window to get windowed signal
    signal_pw = [a * b for a, b in zip(rawBOLD_norm, W)]
    
    #END MATCHING
    y_first = signal_pw[0] 
    y_last = signal_pw[-1]
    slope_ends = (y_last - y_first)/(N-1)
    y_int = y_last - slope_ends*N
    x_values = np.arange(1,N+1)
    #line equation is line connecting first and last points of signal_pw
    line_eq = slope_ends * x_values + y_int
    
    #BRIDGE DETRENDING
    #subtract transposed line equation from signal_pw
    signal_em1 = signal_pw - line_eq[:, np.newaxis]

    #PARAMETERS FOR FFT
    #range1 is first half of signal
    range1 = math.ceil((N+1)/2)
    #frequency vector is sample freq times (0-range1)/N
    freq_row_vector = fs * np.arange(0,range1)/N
    #transpose freq_row_vector to get freq in column format
    freq = freq_row_vector[:,np.newaxis]
    #signal_em1 is the signal after lowPSDwe has been applied, use this signal for PSD
    
    #FFT
    #estimate the power spectral amplitude function A(f)^2 from FFT algorithm on X(t)
    #plot of log(power) vs. log(frequency) needs to be linear over a 2-decade range 
    fftSignal = np.fft.fft(signal_em1,N)  #using fft function, also many others could use? rfft?
    #take first half of signal because symmetric
    fftSignal1 = fftSignal[0:range1]  #not sure if this signal is actually symmetric
    PSD = np.power(abs(fftSignal1),2)/N
    if N%2:   #not sure why multiply by 2
        PSD[2:] = PSD[2:]*2
    else:
        PSD[2:-1] = PSD[2:-1]*2
    
    #GETTING LINEAR PART OF POWER SPECTRUM
    #need a conditition that determines which part is linear -> here we assume between 0.08-0.16Hz
    #the log(power) vs. log(freq) plot is linear from freq 0.08 to 0.16
    #find index of freq 0.08 and 0.16
    min_index1 = np.argmin(abs(freq-0.08)) 
    min_index2 = np.argmin(abs(freq-0.16))
    #splice PSD and freq arrays to have only values in indices
    PSD_linear = PSD[min_index1:min_index2:1]
    freq_linear = freq[min_index1:min_index2:1]
    #take log of linear PSD and freq parts 
    logPSD = np.log10(PSD_linear)
    logfreq = np.log10(freq_linear)
    #power spectrum density plot
    #plt.plot(logfreq,logPSD) 
    #remove all the inf and nan values 
    logPSD_c = np.ma.masked_invalid(logPSD)
    logfreq_c = np.ma.masked_invalid(logfreq)
    
    #GETTING BETA
    #if all values are not infinity, then signal is not noise?
    if(len(logfreq_c)!=0 and len(logfreq_c)!=0):
        print('Signal is not noise')
        #can now find Beta (spectral index)
        #Beta is the slope of the linear regression line of the PSD
        logfreq_c_col = logfreq_c.reshape((-1,1)) #need to reshape x-axis to 2D array to fit regression
        linear_model = LinearRegression().fit(logfreq_c_col,logPSD_c)
        Beta = linear_model.coef_[0,0] #outputs the slope of model
        print('Beta = ', Beta)
    else: 
        #voxel lies outside of the brain
        print('Signal is noise, do not proceed with fractal analysis')
        abort = True
    
    
    #DISPERSIONAL ANALYSIS (DISP)
    #undertake disp analysis if signal is fractional Gaussian Noise (fGn)
    if(Beta>-1 and Beta<0.38 and abort == False):
        print('Signal is fGn, do disperional analysis')
        maxBins = math.ceil(math.log2(abs(len(rawBOLD)))) - 1 
        signal_2 = rawBOLD[0:2**maxBins] #signal now has length of 2^7 (128)
        #initialize empty lists for for-loop
        DISP = np.zeros(maxBins)
        tau = np.zeros(maxBins)
        
        for i in range(0,maxBins): 
                m = 2 ** (i+1) 
                signal_binned = signal_2.reshape(m, int(len(signal_2)/m))
                #get array of mean of each column 
                mean_binned = np.mean(signal_binned, axis=1) 
                #get standard deviation of array of means
                DISP[i] = np.std(mean_binned)
                tau[i] = m 
                if(DISP[i] == 0):
                    DISP[i] = 0.0001
        # take log and omit the first two iterations (m = 2^1 and 2^2)            
        logDISP = np.log10(DISP[2:8]) #why 8? Think it should go until the end of list so works for any input
        logtau = np.log10(tau[2:8])
        #find slope of log log plot
        logtau_col = logtau.reshape((-1,1)) 
        linear_model_DISP = LinearRegression().fit(logtau_col,logDISP)
        #Hurst is slope of log log plot plus 1
        Hurst_fGn_DISP = linear_model_DISP.coef_[0] + 1 
        Hurst = Hurst_fGn_DISP
        print('Hurst = ', Hurst)
        
    

    return Hurst


In [444]:
#run this cell to test Hurst function 
#cell loads data files and makes function call 

#find nifti files to test function with
dir = os.path.join(os.getcwd(), 'sub-08_ses-1/func')
file_csf = os.path.join(dir, 'csf_mask.nii.gz')
file_grey = os.path.join(dir, 'grey_mask.nii.gz')
file_white = os.path.join(dir, 'white_mask.nii.gz')
file_noise = os.path.join(dir, 'noise_mask.nii.gz')
file_sub = os.path.join(dir, 'sub-08_ses-1_task-ins_bold.nii.gz')

#load files into notebook
csfmask = nib.load(file_csf).get_fdata()
csflog = csfmask.astype(bool)
greymask = nib.load(file_grey).get_fdata()
greylog = greymask.astype(bool)
whitemask = nib.load(file_white).get_fdata()
whitelog = whitemask.astype(bool)
noisemask = nib.load(file_noise).get_fdata()
noiselog = noisemask.astype(bool)

sub8ins = nib.load(file_sub).get_fdata()

#find random index of array where element is true after applying mask
[row_c,col_c,page_c] = random.choice(np.argwhere(csflog))
[row_g, col_g, page_g] = random.choice(np.argwhere(greylog))
[row_w,col_w,page_w] = random.choice(np.argwhere(whitelog))
[row_n,col_n,page_n] = random.choice(np.argwhere(noiselog))

#get element in sub8ins data at random index
csf_sample = np.squeeze(sub8ins[row_c,col_c,page_c,:])
grey_sample = np.squeeze(sub8ins[row_g,col_g,page_g,:])
white_sample = np.squeeze(sub8ins[row_w,col_w,page_w,:])
noise_sample = np.squeeze(sub8ins[row_n,col_n,page_n,:])


#call to Hurst function
HurstFunction(grey_sample,2)

#call to decade function
#decade(noise_sample)





Sampling frequency = 0.5
Number of timepoints = 174
Mean rawBOLD = 672.7126436781609
Signal is not noise
Beta =  1.1751443080407475


In [416]:
def decade(signal):
    timepoints = int(len(signal))
    fft = np.fft.fft(signal)
    peaks, _ = find_peaks(np.abs(fft)[:timepoints // 2] * 1 / timepoints)
    f = np.linspace(0, timepoints, timepoints)
    q = f[:timepoints // 2]
    decad = math.log(max(q[peaks])/min(q[peaks]),10)
    print('dec is', decad)